In [26]:
import json
import pandas as pd
import datetime
from datetime import datetime, time, timedelta
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


In [2]:
data = json.load(open('ultimate_data_challenge.json'))

In [3]:
data = pd.DataFrame.from_dict(data)

In [4]:
data.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9


In [5]:
data.last_trip_date = pd.to_datetime(data.last_trip_date)

In [6]:

today = np.max(data.last_trip_date)

In [7]:
today

Timestamp('2014-07-01 00:00:00')

In [8]:
window = timedelta(days = 30)

In [9]:
active = pd.DataFrame(data.last_trip_date > today - window)

In [10]:
active.value_counts(normalize=True)

last_trip_date
False             0.6338
True              0.3662
dtype: float64

In [11]:
dummies = pd.get_dummies(data.city)
data = data.join(dummies)

In [12]:
dummies = pd.get_dummies(data.phone)
data = data.join(dummies)

In [13]:
data.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,Astapor,King's Landing,Winterfell,Android,iPhone
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0,0,1,0,0,1
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0,1,0,0,1,0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0,1,0,0,0,1
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9,0,1,0,0,1
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9,0,0,1,1,0


In [14]:
active.head()

,last_trip_date
0,True
1,False
2,False
3,True
4,False


In [15]:
X = data.drop(['city', 'phone', 'signup_date', 'last_trip_date'], axis=1)

In [16]:
X = X.fillna(0)

In [17]:
corrs = X.corrwith(active.last_trip_date)
corrs

trips_in_first_30_days    0.214728
avg_rating_of_driver      0.146989
avg_surge                -0.003567
surge_pct                 0.010612
ultimate_black_user       0.210259
weekday_pct               0.026413
avg_dist                 -0.090279
avg_rating_by_driver     -0.006596
Astapor                  -0.179373
King's Landing            0.268272
Winterfell               -0.046985
Android                  -0.231731
iPhone                    0.231391
dtype: float64

In [18]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X.values, active.last_trip_date)

In [32]:
clf = LogisticRegression(max_iter = 1000)
clf.fit(Xtrain, ytrain)
accuracy_score(clf.predict(Xtest), ytest)

0.72864

In [33]:
print(clf.coef_)

[[ 1.09339455e-01  1.26848804e-01 -1.43296112e-01  4.40361097e-03
   8.85116901e-01  9.39285387e-04 -3.28170379e-02 -9.26471441e-02
  -1.05644192e+00  7.00345049e-01 -4.84273634e-01 -5.11671650e-01
   6.13768006e-01]]


In [34]:
print(classification_report(ytest,clf.predict(Xtest)))

              precision    recall  f1-score   support

       False       0.75      0.86      0.80      7880
        True       0.68      0.50      0.58      4620

    accuracy                           0.73     12500
   macro avg       0.71      0.68      0.69     12500
weighted avg       0.72      0.73      0.72     12500



In [35]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=500)
clf.fit(Xtrain, ytrain)

RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [36]:
print(classification_report(ytest,clf.predict(Xtest)))

              precision    recall  f1-score   support

       False       0.81      0.82      0.82      7880
        True       0.69      0.67      0.68      4620

    accuracy                           0.77     12500
   macro avg       0.75      0.75      0.75     12500
weighted avg       0.77      0.77      0.77     12500

